<a href="https://colab.research.google.com/github/QuantumStar1999/GoogleColabNotebooks/blob/main/kaggle/house_prices_advanced_regression_techniques_linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, json, IPython
project_name = "house-prices-advanced-regression-techniques"
token = {"username":"santanukundu","key":"de462df622bbe0817a45f17f3c1373eb"}
os.environ['KAGGLE_CONFIG_DIR']='.'
with open('kaggle.json', "w") as f:
    json.dump(token, f)
!chmod 600 ./kaggle.json
!kaggle competitions download -c $project_name
project_name += ".zip"
!unzip $project_name && rm $project_name
IPython.display.clear_output()

In [2]:
!cat data_description.txt

MSSubClass: Identifies the type of dwelling involved in the sale.	

        20	1-STORY 1946 & NEWER ALL STYLES
        30	1-STORY 1945 & OLDER
        40	1-STORY W/FINISHED ATTIC ALL AGES
        45	1-1/2 STORY - UNFINISHED ALL AGES
        50	1-1/2 STORY FINISHED ALL AGES
        60	2-STORY 1946 & NEWER
        70	2-STORY 1945 & OLDER
        75	2-1/2 STORY ALL AGES
        80	SPLIT OR MULTI-LEVEL
        85	SPLIT FOYER
        90	DUPLEX - ALL STYLES AND AGES
       120	1-STORY PUD (Planned Unit Development) - 1946 & NEWER
       150	1-1/2 STORY PUD - ALL AGES
       160	2-STORY PUD - 1946 & NEWER
       180	PUD - MULTILEVEL - INCL SPLIT LEV/FOYER
       190	2 FAMILY CONVERSION - ALL STYLES AND AGES

MSZoning: Identifies the general zoning classification of the sale.
		
       A	Agriculture
       C	Commercial
       FV	Floating Village Residential
       I	Industrial
       RH	Residential High Density
       RL	Residential Low Density
       RP	Residential Low Density Park 
       RM

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [7]:
class HousePricePrediction():

    def __init__(self):
        self.train = pd.read_csv('train.csv')
        self.test = pd.read_csv('test.csv')
        self.numerical_features   = list(self.train.select_dtypes(exclude='object').columns)
        self.categorical_features = list(self.train.select_dtypes(include='object').columns)
        self.dropping_cols = ['Id']
        self.target = 'SalePrice'

        self.test_size = 0.2
        self.model = LinearRegression()

        self.predicted_data = pd.DataFrame()

        self.saved_file_name = 'output.csv'

        self.pipeline()

    """
    preprocessing includes
    1. categorical to numerical conversion
    2. save mapper object so we can use it for test_data
    3. save scaler objects
    """
    def preprocess(self, df):
        calc_cols = ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt']
        for i in calc_cols:
            df[i] = df['YrSold'] - df[i]

        df['YrSold'] = 2023 - df['YrSold']

        # filling na values
        # numerical cols will be filled with 0
        # categorical ones will be filled with 'Not present'
        self.mapper_na_to_val = dict()
        for i in self.numerical_features:
            self.mapper_na_to_val[i] = 0
        for i in self.categorical_features:
            self.mapper_na_to_val[i]='Not present'

        df = df.fillna(value= self.mapper_na_to_val)
        return df

    def encoding(self,df):
        self.categorical_mapper = {}
        for i in self.categorical_features:
            self.categorical_mapper[i] = dict(df.groupby(i)['SalePrice'].mean())
            df[i] = df.groupby(i)['SalePrice'].transform('mean')
            if 'Not present' not in self.categorical_mapper[i].keys():
                self.categorical_mapper[i]['Not present'] = df['SalePrice'].mean()
        return df

    def training(self,df):
        """"
        Drop columns if required
        """
        self.scaler_X, self.scaler_Y = MinMaxScaler(), MinMaxScaler()

        y = df[[self.target]]
        X = df.drop([self.target] + self.dropping_cols ,axis=1)
        self.cols = X.columns

        X, y = self.scaler_X.fit_transform(X), self.scaler_Y.fit_transform(y)

        X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=self.test_size, random_state=42)

        self.model.fit(X_train, y_train)

        print("R2 score for training data: ", self.model.score(X_train, y_train))
        print("R2 score for testing data: ", self.model.score(X_test, y_test))

    def prediction(self,df):
        x = self.preprocess(df).replace(self.categorical_mapper).drop(self.dropping_cols, axis=1)

        self.predicted_data['Id'] = df['Id']
        self.predicted_data[self.target] = self.scaler_Y.inverse_transform(
            self.model.predict(self.scaler_X.transform(x))
            )
        self.predicted_data.to_csv(self.saved_file_name,index=False)

    def pipeline(self):
        self.training(self.encoding(self.preprocess(self.train)))
        self.prediction(self.test)
    def coef_(self):
        return pd.DataFrame(zip(self.cols,self.model.coef_.flatten()), columns= ['feature_name', 'weight']
                    ).sort_values('weight', ascending=False, key= lambda x: abs(x)).reset_index(drop=True)
    def intercept_(self):
        return self.model.intercept_

In [8]:
prediction = HousePricePrediction()

R2 score for training data:  0.8752341843265437
R2 score for testing data:  0.8740185708941218


In [9]:
prediction.coef_()

,feature_name,weight
0,GrLivArea,-1.416874e+12
1,1stFlrSF,1.163289e+12
2,2ndFlrSF,5.512143e+11
3,TotalBsmtSF,4.882365e+11
4,BsmtFinSF1,-4.509995e+11
...,...,...
74,CentralAir,1.269218e-03
75,BsmtFinType1,1.046488e-03
76,FireplaceQu,7.548733e-04
77,PavedDrive,2.989690e-04
